In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd Data

/content/drive/MyDrive/Data


In [5]:
import pandas as pd
non_PRC = pd.read_csv('stand/stand.csv')
PRC = pd.read_csv('stand/stand_par.csv')
clinical_assessment = pd.read_excel('Clinical Assessment.xlsx')

In [6]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import warnings
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [7]:
score = clinical_assessment.iloc[:,9]

In [8]:
del score[0]
del score[1]
del score[2]
del score[3]
del score[7]
del score[9]
del score[10]
del score[13]
del score[15]
del score[16]
del score[17]
del score[18]
del score[19]
del score[20]
del score[21]
del score[22]
del score[23]
del score[24]
del score[25]
del score[26]
del score[27]
del score[31]
del score[34]
del score[36]
del score[39]
del score[44]
del score[45]


In [9]:
y = []
for value in score:
    if value >= 45:
        y.append(1)
    else:
        y.append(0)
print(y)

[1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Non-paretic Side

In [10]:
df_non = pd.DataFrame(non_PRC)
y = pd.DataFrame(y)
non = pd.concat([y,df_non],axis=1)
colname = list(non.columns)
colname[0] = "score"
non.columns = colname
non.columns

Index(['score', 'Unnamed: 0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7',
       'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16',
       'PC17', 'PC18', 'PC19'],
      dtype='object')

In [11]:
X = non.loc[:,['PC2','PC10','PC15']]
y = non['score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123, stratify=y)

### logistic regression

In [12]:
lr_model = LogisticRegression()

# 선택된 특성으로 모델 학습
lr_model.fit(X_train, y_train)

accuracy_scores = cross_val_score(lr_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(lr_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(lr_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(lr_model, X_test, y_test, scoring='roc_auc', cv=2)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.875
Average Precision: 0.875
Average F1-score: 0.9285
Average AUC: 1.0


### Random Forest

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=2)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Average Accuracy: 0.75
Average Precision: 0.75
Average F1-score: 0.857
Average AUC: 0.0


### SVM Linear

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=2, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=2)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.875
Average Precision: 0.875
Average F1-score: 0.9285
Average AUC: 1.0


### SVM Poly

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=2, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=2)

# Display the average performance metrics over the 2 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.75
Average Precision: 0.75
Average F1-score: 0.857
Average AUC: 0.8335


### XGBoost

In [16]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=2)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Average Accuracy: 0.75
Average Precision: 0.75
Average F1-score: 0.857
Average AUC: 0.5


## paretic-Side

In [24]:
df_par = pd.DataFrame(PRC)
y = pd.DataFrame(y)
par = pd.concat([y,df_par],axis=1)
colname = list(par.columns)
colname[0] = "score"

In [25]:
import random
random.seed(1)
X = par.loc[:,['PC7']]
y = par['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123,stratify=y)

### logistic regression

In [26]:
lr_model = LogisticRegression()

# 선택된 특성으로 모델 학습
lr_model.fit(X_train, y_train)

accuracy_scores = cross_val_score(lr_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(lr_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(lr_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(lr_model, X_test, y_test, scoring='roc_auc', cv=2)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.875
Average Precision: 0.875
Average F1-score: 0.9285
Average AUC: 1.0


### paretic RandomForest

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=2)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Average Accuracy: 0.875
Average Precision: 0.875
Average F1-score: 0.9285
Average AUC: 1.0


### paretic SVM Linear

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=2, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=2)

# Display the average performance metrics over the 2 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.875
Average Precision: 0.875
Average F1-score: 0.9285
Average AUC: 1.0


### paretic SVM Poly

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=2, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=2)

# Display the average performance metrics over the 2 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.75
Average Precision: 0.875
Average F1-score: 0.8285
Average AUC: 1.0


### paretic XGBoost

In [30]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 2],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=2, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=2)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=2)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=2)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=2)

# Display the average performance metrics over the 2 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.75
Average Precision: 0.75
Average F1-score: 0.857
Average AUC: 0.5
